In [10]:
import cv2
import numpy as np

path="images/test1.jpg"
Q_N=5
C_N=5
COLS=1
Q_N_percol=int(Q_N/COLS)
HEIGHT=500-(500%Q_N)
WIDTH=500-(500%C_N)
TRESH=200
CAMERA=True


ANS=np.array([1 for _ in range(Q_N)])

ANS=ANS.reshape(COLS,Q_N_percol)

In [11]:
def get_points(img):
    i = 0
    myPointsNew = np.zeros((4, 1, 2), dtype=np.int32)
    def click_event(event, x, y, flags, param):
        nonlocal i, myPointsNew  
        if event == cv2.EVENT_LBUTTONDOWN and i < 4:  
            cv2.circle(img, (x, y), 3, (0, 0, 255), -1)
            myPointsNew[i] = [[x, y]]
            i += 1
            cv2.imshow('Cor', img)

    cv2.imshow('Cor', img)
    cv2.setMouseCallback('Cor', click_event)

    while True:
        key = cv2.waitKey(20)
        if key == 27 or i >= 4:  
            break
            
    cv2.destroyWindow("Cor")
    return reorder(myPointsNew)


def reorder(myPoints):
 
    myPoints = myPoints.reshape((4, 2))
    myPointsNew = np.zeros((4, 1, 2), dtype=np.int32)
    add = myPoints.sum(1)
 
    myPointsNew[0] = myPoints[np.argmin(add)]
    myPointsNew[3] =myPoints[np.argmax(add)]
    diff = np.diff(myPoints, axis=1)
    myPointsNew[1] =myPoints[np.argmin(diff)]
    myPointsNew[2] = myPoints[np.argmax(diff)]
 
    return myPointsNew


def get_view(biggest):
  pts1 = np.float32(biggest) 
  pts2 = np.float32([[0, 0],[WIDTH, 0], [0, HEIGHT],[WIDTH, HEIGHT]]) 
  matrix = cv2.getPerspectiveTransform(pts1, pts2)
  return matrix


def splitBoxes(img):
    rows = np.vsplit(img, Q_N_percol)
    boxes = []
    
    for r in rows:
        cols = np.hsplit(r, C_N)
        boxes.append(cols)
    return boxes


def showAnswers(img, myans, ans):
    secW = int(img.shape[1] / C_N)
    secH = int(img.shape[0] / Q_N_percol)
    grading=(myans==ans).astype(float)
    for x in range(0, Q_N_percol):
        myAns = myans[x]-1
        cX = (myAns * secW) + secW // 2
        cY = (x * secH) + secH // 2
        if grading[x] == 1:
            myColor = (0, 255, 0)
            cv2.circle(img, (cX, cY), 15, myColor, -1)
        else:
            myColor = (0, 0, 255)
            cv2.circle(img, (cX, cY), 15, myColor, -1)

        # CORRECT ANSWER
        # myColor = (0, 255, 0)
        # correctAns = (ans[x])-1
        # cv2.circle(img, ((correctAns * secW) + secW // 2, (x * secH) + secH // 2),
        #            7, myColor, -1)


def drawGrid(img):
    secW = int(img.shape[1] / C_N)
    secH = int(img.shape[0] / Q_N_percol)
    for i in range(0, Q_N_percol+1):
        pt1 = (0, secH * i)
        pt2 = (img.shape[1], secH * i)
        cv2.line(img, pt1, pt2, (255, 255, 0), 3)
    for i in range(0,C_N+1):
        pt3 = (secW * i, 0)
        pt4 = (secW * i, img.shape[0])
        cv2.line(img, pt3, pt4, (255, 255, 0), 3)
        

def stackImages(imgArray,scale,lables=[]):
    rows = len(imgArray)
    cols = len(imgArray[0])
    rowsAvailable = isinstance(imgArray[0], list)
    width = imgArray[0][0].shape[1]
    height = imgArray[0][0].shape[0]
    if rowsAvailable:
        for x in range ( 0, rows):
            for y in range(0, cols):
                imgArray[x][y] = cv2.resize(imgArray[x][y], (0, 0), None, scale, scale)
                if len(imgArray[x][y].shape) == 2: imgArray[x][y]= cv2.cvtColor( imgArray[x][y], cv2.COLOR_GRAY2BGR)
        imageBlank = np.zeros((height, width, 3), np.uint8)
        hor = [imageBlank]*rows
        hor_con = [imageBlank]*rows
        for x in range(0, rows):
            hor[x] = np.hstack(imgArray[x])
            hor_con[x] = np.concatenate(imgArray[x])
        ver = np.vstack(hor)
        ver_con = np.concatenate(hor)
    else:
        for x in range(0, rows):
            imgArray[x] = cv2.resize(imgArray[x], (0, 0), None, scale, scale)
            if len(imgArray[x].shape) == 2: imgArray[x] = cv2.cvtColor(imgArray[x], cv2.COLOR_GRAY2BGR)
        hor= np.hstack(imgArray)
        hor_con= np.concatenate(imgArray)
        ver = hor
    if len(lables) != 0:
        eachImgWidth= int(ver.shape[1] / cols)
        eachImgHeight = int(ver.shape[0] / rows)
        for d in range(0, rows):
            for c in range (0,cols):
                cv2.rectangle(ver,(c*eachImgWidth,eachImgHeight*d),(c*eachImgWidth+len(lables[d][c])*13+27,30+eachImgHeight*d),(255,255,255),cv2.FILLED)
                cv2.putText(ver,lables[d][c],(eachImgWidth*c+10,eachImgHeight*d+20),cv2.FONT_HERSHEY_COMPLEX,0.7,(255,0,255),2)
    return ver


def update_thresh(value):
    global TRESH
    # Get current brightness value from slider
    TRESH = cv2.getTrackbarPos('thresh', 'Image') 

def add_images(img,ans):
    mask = cv2.cvtColor(ans, cv2.COLOR_BGR2GRAY)
    _, mask = cv2.threshold(mask, 30, 1.0, cv2.THRESH_BINARY) 


    mask = cv2.merge([mask, mask, mask])


    combined = ans * mask + img * (1 - mask)
    return combined

In [12]:

def grade(img):
  img=cv2.resize(img,(WIDTH,HEIGHT))
  gradeimage=img.copy()
  ansimage=img.copy()
  output=img.copy()

  #Anserws percpective
  myans=[]
  colsoutput=[]
  for i in range(COLS):
    #Get Percpetive 
    myPointsNew=get_points(ansimage)
    matrix=get_view(myPointsNew)
    imgWarpColored = cv2.warpPerspective(ansimage, matrix, (WIDTH, HEIGHT))

    #Binary
    imgWarpColored1=cv2.cvtColor(imgWarpColored,cv2.COLOR_BGR2GRAY)
    imgWarpColored1=cv2.GaussianBlur(imgWarpColored1,(5,5),0)
    imgWarpColored1=cv2.threshold(imgWarpColored1,TRESH,255,cv2.THRESH_BINARY_INV)[1]
    split=splitBoxes(imgWarpColored1)

    #User Answer 
    myPixelVal = np.zeros((int(Q_N_percol), C_N),dtype=int) 
    for row in range(len(split)):
      for col in range(len(split[0])):
        totalPixels = cv2.countNonZero(split[row][col])
        myPixelVal[row,col]=totalPixels
    ans=list(np.array([int(np.argmax(myPixelVal[i])+1) for i in range(Q_N_percol)]))
    myans.append(ans)
    # Draw Answers and Grid and Return
    show_answers=np.zeros_like(imgWarpColored)
    showAnswers(show_answers,ans,ANS[i])
    drawGrid(show_answers)
    imgInvWarp = cv2.warpPerspective(show_answers, np.linalg.inv(matrix), (WIDTH, HEIGHT))
    colsoutput.append(imgInvWarp)
    cv2.imshow(f'col {i+1}',stackImages([show_answers,imgWarpColored1,imgWarpColored],.7))

  #Score
  allans=[i for inner in myans for i in inner]
  allans=np.array(allans)
  score=np.sum((allans==ANS.reshape(-1)).astype(float))/Q_N

  #Grade percpective
  myPointsNew=get_points(gradeimage)
  matrixg=get_view(myPointsNew)
  gradwarp=cv2.warpPerspective(gradeimage,matrixg, (WIDTH, HEIGHT))
  imgRawGrade = np.zeros_like(gradwarp, np.uint8)  
  cv2.putText(imgRawGrade, str(float(score*100)) + "%", (50, 200),
              cv2.FONT_HERSHEY_COMPLEX, 5, (0,255,0), 5)  
  imgInvGradeDisplay = cv2.warpPerspective(imgRawGrade, np.linalg.inv(matrixg), (WIDTH, HEIGHT))


  #Combine
  output = add_images(output,imgInvGradeDisplay)
  for i in colsoutput:
    output = add_images(output,i)


  cv2.imshow("output",output)
  cv2.waitKey(0)
  cv2.destroyAllWindows()


In [ ]:
if CAMERA :
    cap = cv2.VideoCapture(0)
    cv2.namedWindow('Image')
    cv2.createTrackbar('thresh', 'Image', 0, 300, update_thresh)
    while True:
        ret, frame = cap.read()
        
        binnary=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        binnary=cv2.GaussianBlur(binnary,(5,5),0)
        binnary=cv2.threshold(binnary,TRESH,255,cv2.THRESH_BINARY_INV)[1]
        cv2.imshow("live",binnary)

        if cv2.waitKey(1) & 0xFF == ord('g'):
            grade(frame)
            cv2.namedWindow('Image')
            cv2.createTrackbar('thresh', 'Image', 0, 300, update_thresh)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
else:
    grade(cv2.imread(path))
